In [ ]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import statistics
import scipy.stats as st
import datetime
import plotly
import plotly.graph_objects as go

In [ ]:
filepath='/Data/'

CtyConfOri=pd.read_csv(filepath+'ArrivalTime/'+'County_ArrT_allrecords_Th1_tillJuly.csv')

CtyConfOri['FIPS']=CtyConfOri['FIPS'].astype(int)
ArrT=CtyConfOri[['FIPS','Days']]
ArrT.columns=['FIPS','Daysfrom122']
CtySource=6075
CtyDayZero=CtyConfOri[CtyConfOri['FIPS']==CtySource]['Days'].iat[0]

ArrT['Days']=ArrT['Daysfrom122']-CtyDayZero
ACS=pd.read_csv(filepath+'US_County_ACS_2018.csv')

In [ ]:
CtyLoc=CtyConfOri[['FIPS','Admin2','Province_State','Lat','Long_']]

In [ ]:
FileName='Src6075_25025_EF_LN_NoBase_DevCnt_AllCty_AllTime_tillJune.csv'
vhat=0.00609056
vhat_low=0.0059
vhat_high=0.0063
CtySource=6075

SecondSource=25025
SingleSrc=pd.read_csv('/GlobalOpt_CountyDynamicMobilityCorrelation/Intermediate_Result_EF/SingleSource/SingleSource'+str(CtySource)+'EFRec.csv')
MultipleSrc1=pd.read_csv('/GlobalOpt_CountyDynamicMobilityCorrelation/Intermediate_Result_EF/MultipleSource/MultipleSource'+str(CtySource)+'_'+str(SecondSource)+'EFRec.csv')
SingleSrc['Src']='Single'
MultipleSrc1['Src']='Multiple'
EFRecOri=SingleSrc.append(MultipleSrc1)

dateStart=datetime.date(2020, 1, 21)
dateEnd=datetime.date(2020, 6, 29)


In [ ]:
import datetime
import matplotlib.ticker as plticker
Daylist=np.sort(EFRecOri['Days'].unique())

In [ ]:
DateZero=datetime.datetime(2020, 1, 21)+datetime.timedelta(days=np.float64(CtyDayZero))

In [ ]:
Sepday=45-CtyDayZero
SepDate=DateZero+datetime.timedelta(days=np.float64(Sepday))

In [ ]:
ATDRec=[]
TsRec=[]
DateRec=[]
SingleTotRec=pd.DataFrame()
MultipleTotRec=pd.DataFrame()
WinLen=7
ArrTCount=[]
DayShift=0
SingleEF=EFRecOri[EFRecOri['Src']=='Single']
MultipleEF=EFRecOri[EFRecOri['Src']=='Multiple']
for Tsday in Daylist[1:-1]: 
    EFRec=SingleEF.copy()
    EFRec.loc[:,'Days']=EFRec['Days']+DayShift
    EFBase=EFRec[(EFRec['Days']>=Tsday-WinLen)&(EFRec['Days']<=Tsday-1)].groupby('Target').mean().reset_index()
    EFAft=EFRec[(EFRec['Days']>=Tsday-WinLen+1)&(EFRec['Days']<=Tsday)].groupby('Target').mean().reset_index()

    EFdiff=EFBase.merge(EFAft,how='inner',left_on='Target',right_on='Target')
    EFdiff['EFdiff']=EFdiff['CapD_y']-EFdiff['CapD_x']
    EFdiff=EFdiff[['Target','EFdiff']]
    EFdiff=EFdiff.merge(ArrT,how='inner',left_on='Target',right_on='FIPS')
    M=len(EFdiff)
    EFdiff['State']=EFdiff['FIPS'].astype(str).str.zfill(5).str[:2].astype(int)
    
    EFdiff.loc[EFdiff['Days']<=Tsday, 'EFdiff'] = 0
    
    EFState=EFdiff[['FIPS','State','EFdiff']]
    TsDate=(DateZero + datetime.timedelta(days=np.float64(Tsday))).strftime('%m-%d')
    EFState['Date']=(DateZero + datetime.timedelta(days=np.float64(Tsday))).date()
    EFState['DateStr']=TsDate
    EFState['Day']=Tsday
    SingleTotRec=SingleTotRec.append(EFState)
    
for Tsday in Daylist[1:-1]: 
    EFRec=MultipleEF.copy()
    EFRec.loc[:,'Days']=EFRec['Days']+DayShift
    EFBase=EFRec[(EFRec['Days']>=Tsday-WinLen)&(EFRec['Days']<=Tsday-1)].groupby('Target').mean().reset_index()
    EFAft=EFRec[(EFRec['Days']>=Tsday-WinLen+1)&(EFRec['Days']<=Tsday)].groupby('Target').mean().reset_index()

    EFdiff=EFBase.merge(EFAft,how='inner',left_on='Target',right_on='Target')
    EFdiff['EFdiff']=EFdiff['CapD_y']-EFdiff['CapD_x']
    EFdiff=EFdiff[['Target','EFdiff']]
    EFdiff=EFdiff.merge(ArrT,how='inner',left_on='Target',right_on='FIPS')
    M=len(EFdiff)
    EFdiff['State']=EFdiff['FIPS'].astype(str).str.zfill(5).str[:2].astype(int)
    
    EFdiff.loc[EFdiff['Days']<=Tsday, 'EFdiff'] = 0
    EFState=EFdiff[['FIPS','State','EFdiff']]
    TsDate=(DateZero + datetime.timedelta(days=np.float64(Tsday))).strftime('%m-%d')
    EFState['Date']=(DateZero + datetime.timedelta(days=np.float64(Tsday))).date()
    EFState['DateStr']=TsDate
    EFState['Day']=Tsday
    MultipleTotRec=MultipleTotRec.append(EFState)

In [ ]:
TotRecOri=SingleTotRec[(SingleTotRec['Day']<=Sepday+DayShift)].append(MultipleTotRec[(MultipleTotRec['Day']>Sepday+DayShift)])
TotRecOri.head(3)
for st in list(set(TotRecOri['FIPS'])):
    TotRecOri.loc[TotRecOri['FIPS']==st,'CumuEFdiff']=TotRecOri[TotRecOri['FIPS']==st]['EFdiff'].cumsum()
TotRecOri['ATD']=TotRecOri['CumuEFdiff']*vhat
TotRecOri['ATD_low']=TotRecOri['CumuEFdiff']*vhat_low
TotRecOri['ATD_high']=TotRecOri['CumuEFdiff']*vhat_high

In [ ]:
ATDRec=[]
TsRec=[]
DateRec=[]
SingleTotRec=pd.DataFrame()
MultipleTotRec=pd.DataFrame()
WinLen=7
ArrTCount=[]

DayShift=-7 

SingleEF=EFRecOri[EFRecOri['Src']=='Single']
MultipleEF=EFRecOri[EFRecOri['Src']=='Multiple']
for Tsday in Daylist[1:-1]:
    EFRec=SingleEF.copy()
    EFRec.loc[:,'Days']=EFRec['Days']+DayShift
    EFBase=EFRec[(EFRec['Days']>=Tsday-WinLen)&(EFRec['Days']<=Tsday-1)].groupby('Target').mean().reset_index()
    EFAft=EFRec[(EFRec['Days']>=Tsday-WinLen+1)&(EFRec['Days']<=Tsday)].groupby('Target').mean().reset_index()

    EFdiff=EFBase.merge(EFAft,how='inner',left_on='Target',right_on='Target')
    EFdiff['EFdiff']=EFdiff['CapD_y']-EFdiff['CapD_x']
    EFdiff=EFdiff[['Target','EFdiff']]
    EFdiff=EFdiff.merge(ArrT,how='inner',left_on='Target',right_on='FIPS')
    M=len(EFdiff)
    EFdiff['State']=EFdiff['FIPS'].astype(str).str.zfill(5).str[:2].astype(int)
    
    EFdiff.loc[EFdiff['Days']<=Tsday, 'EFdiff'] = 0

    EFState=EFdiff[['FIPS','State','EFdiff']]
    TsDate=(DateZero + datetime.timedelta(days=np.float64(Tsday))).strftime('%m-%d')
    EFState['Date']=(DateZero + datetime.timedelta(days=np.float64(Tsday))).date()
    EFState['DateStr']=TsDate
    EFState['Day']=Tsday
    SingleTotRec=SingleTotRec.append(EFState)
    
for Tsday in Daylist[1:-1]:
    EFRec=MultipleEF.copy()
    EFRec.loc[:,'Days']=EFRec['Days']+DayShift
    EFBase=EFRec[(EFRec['Days']>=Tsday-WinLen)&(EFRec['Days']<=Tsday-1)].groupby('Target').mean().reset_index()
    EFAft=EFRec[(EFRec['Days']>=Tsday-WinLen+1)&(EFRec['Days']<=Tsday)].groupby('Target').mean().reset_index()

    EFdiff=EFBase.merge(EFAft,how='inner',left_on='Target',right_on='Target')
    EFdiff['EFdiff']=EFdiff['CapD_y']-EFdiff['CapD_x']
    EFdiff=EFdiff[['Target','EFdiff']]
    EFdiff=EFdiff.merge(ArrT,how='inner',left_on='Target',right_on='FIPS')
    M=len(EFdiff)
    EFdiff['State']=EFdiff['FIPS'].astype(str).str.zfill(5).str[:2].astype(int)
    
    EFdiff.loc[EFdiff['Days']<=Tsday, 'EFdiff'] = 0
    EFState=EFdiff[['FIPS','State','EFdiff']]
    TsDate=(DateZero + datetime.timedelta(days=np.float64(Tsday))).strftime('%m-%d')
    EFState['Date']=(DateZero + datetime.timedelta(days=np.float64(Tsday))).date()
    EFState['DateStr']=TsDate
    EFState['Day']=Tsday
    MultipleTotRec=MultipleTotRec.append(EFState)

In [ ]:
TotRec=SingleTotRec[(SingleTotRec['Day']<=Sepday+DayShift)].append(MultipleTotRec[(MultipleTotRec['Day']>Sepday+DayShift)])
TotRec.head(3)
for st in list(set(TotRec['FIPS'])):
    TotRec.loc[TotRec['FIPS']==st,'CumuEFdiff']=TotRec[TotRec['FIPS']==st]['EFdiff'].cumsum()
TotRec['ATD']=TotRec['CumuEFdiff']*vhat
TotRec['ATD_low']=TotRec['CumuEFdiff']*vhat_low
TotRec['ATD_high']=TotRec['CumuEFdiff']*vhat_high

In [ ]:
Totdiff=TotRecOri.merge(TotRec,on=['FIPS','DateStr','Day','Date'],suffixes=('_Ori','_shft'))

In [ ]:
Totdiff['ATD']=Totdiff['ATD_shft']-Totdiff['ATD_Ori']
Totdiff['ATD_low']=Totdiff['ATD_low_shft']-Totdiff['ATD_low_Ori']
Totdiff['ATD_high']=Totdiff['ATD_high_shft']-Totdiff['ATD_high_Ori']

In [ ]:
Totdiff=Totdiff.merge(ArrT,on='FIPS')

In [ ]:
datestr='04-03'
inpdata=Totdiff[Totdiff['DateStr']==datestr]
inpdata=inpdata.merge(CtyLoc,left_on='FIPS',right_on='FIPS',how='left')
inpdata['text']=inpdata['Admin2']+','+inpdata['Province_State']
inpdata['ATD'].hist()

In [ ]:
ATDRec=[]
TsRec=[]
DateRec=[]
SingleTotRec=pd.DataFrame()
MultipleTotRec=pd.DataFrame()
WinLen=7
ArrTCount=[]

DayShift=7

SingleEF=EFRecOri[EFRecOri['Src']=='Single']
MultipleEF=EFRecOri[EFRecOri['Src']=='Multiple']

for Tsday in Daylist[1:-1]:
    EFRec=SingleEF.copy()
    EFRec.loc[:,'Days']=EFRec['Days']+DayShift

    EFBase=EFRec[(EFRec['Days']>=Tsday-WinLen)&(EFRec['Days']<=Tsday-1)].groupby('Target').mean().reset_index()
    EFAft=EFRec[(EFRec['Days']>=Tsday-WinLen+1)&(EFRec['Days']<=Tsday)].groupby('Target').mean().reset_index()

    EFdiff=EFBase.merge(EFAft,how='inner',left_on='Target',right_on='Target')
    EFdiff['EFdiff']=EFdiff['CapD_y']-EFdiff['CapD_x']
    EFdiff=EFdiff[['Target','EFdiff']]
    EFdiff=EFdiff.merge(ArrT,how='inner',left_on='Target',right_on='FIPS')
    
    M=len(EFdiff)
    EFdiff['State']=EFdiff['FIPS'].astype(str).str.zfill(5).str[:2].astype(int)
    
    
    EFdiff.loc[EFdiff['Days']<=Tsday, 'EFdiff'] = 0

    EFState=EFdiff[['FIPS','State','EFdiff']]
    TsDate=(DateZero + datetime.timedelta(days=np.float64(Tsday))).strftime('%m-%d')
    EFState['Date']=(DateZero + datetime.timedelta(days=np.float64(Tsday))).date()
    EFState['DateStr']=TsDate
    EFState['Day']=Tsday
    SingleTotRec=SingleTotRec.append(EFState)
    
for Tsday in Daylist[1:-1]:
    EFRec=MultipleEF.copy()
    EFRec.loc[:,'Days']=EFRec['Days']+DayShift

    EFBase=EFRec[(EFRec['Days']>=Tsday-WinLen)&(EFRec['Days']<=Tsday-1)].groupby('Target').mean().reset_index()
    EFAft=EFRec[(EFRec['Days']>=Tsday-WinLen+1)&(EFRec['Days']<=Tsday)].groupby('Target').mean().reset_index()

    EFdiff=EFBase.merge(EFAft,how='inner',left_on='Target',right_on='Target')
    EFdiff['EFdiff']=EFdiff['CapD_y']-EFdiff['CapD_x']
    EFdiff=EFdiff[['Target','EFdiff']]
    EFdiff=EFdiff.merge(ArrT,how='inner',left_on='Target',right_on='FIPS')
    M=len(EFdiff)
    EFdiff['State']=EFdiff['FIPS'].astype(str).str.zfill(5).str[:2].astype(int)
    
    EFdiff.loc[EFdiff['Days']<=Tsday, 'EFdiff'] = 0

    EFState=EFdiff[['FIPS','State','EFdiff']]
    TsDate=(DateZero + datetime.timedelta(days=np.float64(Tsday))).strftime('%m-%d')
    EFState['Date']=(DateZero + datetime.timedelta(days=np.float64(Tsday))).date()
    EFState['DateStr']=TsDate
    EFState['Day']=Tsday
    MultipleTotRec=MultipleTotRec.append(EFState)
       
TotRecDelay=SingleTotRec[(SingleTotRec['Day']<=Sepday+DayShift)].append(MultipleTotRec[(MultipleTotRec['Day']>Sepday+DayShift)])
TotRecDelay.head(3)
for st in list(set(TotRecDelay['FIPS'])):
    TotRecDelay.loc[TotRecDelay['FIPS']==st,'CumuEFdiff']=TotRecDelay[TotRecDelay['FIPS']==st]['EFdiff'].cumsum()
TotRecDelay['ATD']=TotRecDelay['CumuEFdiff']*vhat
TotRecDelay['ATD_low']=TotRecDelay['CumuEFdiff']*vhat_low
TotRecDelay['ATD_high']=TotRecDelay['CumuEFdiff']*vhat_high
    

In [ ]:
datestr='04-01'
#Totdiff=Totdiffori.copy()
Gamma=pd.read_csv('/Gamma_Dataset.csv',index_col=[0])
Gamma
GammaAdj=Gamma.copy()
# Cty day zero counts from 1/22
GammaAdj['CumGamma']=GammaAdj[GammaAdj['day']>=int(CtyDayZero)+1]['Gamma'].cumsum()

GammaAdj['dayadj']=GammaAdj['day']-int(CtyDayZero)+1

# CountyEffdistRecALL=CountyEffdistRecALL.merge(GammaAdj[['dayadj','CumGamma']],left_on='Days',right_on='dayadj',how='left')
GammaAdj=GammaAdj.fillna(0)
inpdata=TotRecOri[TotRecOri['DateStr']==datestr]
inpdata['x']=inpdata['ATD']
inpdata['x_low']=inpdata['ATD_low']
inpdata['x_high']=inpdata['ATD_high']
inpdata['ATD']=np.interp(inpdata['x'],GammaAdj['CumGamma'],GammaAdj['dayadj'])
inpdata['ATD_low']=np.interp(inpdata['x_low'],GammaAdj['CumGamma'],GammaAdj['dayadj'])
inpdata['ATD_high']=np.interp(inpdata['x_high'],GammaAdj['CumGamma'],GammaAdj['dayadj'])

inpdata=inpdata.merge(CtyLoc,left_on='FIPS',right_on='FIPS',how='left')
inpdata['text']=inpdata['Admin2']+','+inpdata['Province_State']+','+inpdata['FIPS'].astype(str)#+','+inpdata['Days'].astype(str)
inpdata['FIPS_zfill']=inpdata['FIPS'].astype(str).str.zfill(5)
from urllib.request import urlopen
import json
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)

    
import plotly.express as px

fig = px.choropleth(inpdata, geojson=counties, locations='FIPS_zfill', color='ATD',
                           color_continuous_scale='GnBu',#"Viridis",
                           range_color=(0, 15),
                           #scope="usa",
                           labels={'ATD':'Arrival Time Delay'}
                          )    
    

fig.update_layout(
        #title = 'Arrival time delay for every counties on '+datestr+' if national emergency declares seven days earlier',
        paper_bgcolor = 'rgba(0,0,0,0)',
        plot_bgcolor = 'rgba(0,0,0,0)',
        margin=dict(l=0, r=0, t=0, b=0),
        width=1300,height=700,
        geo_scope='usa',
        coloraxis_showscale=False
    )
#fig.update_traces(showscale=False)
fig.write_image('/Figure/FinalResult/ATD_Map/ATDMap_Ori_Fill407.pdf')
fig.write_image('/Figure/FinalResult/ATD_Map/ATDMap_Ori_Fill407.png')



In [ ]:
datestr='04-01'
Gamma=pd.read_csv('/Gamma_Dataset.csv',index_col=[0])
Gamma
GammaAdj=Gamma.copy()
# Cty day zero counts from 1/22
GammaAdj['CumGamma']=GammaAdj[GammaAdj['day']>=int(CtyDayZero)+1]['Gamma'].cumsum()

GammaAdj['dayadj']=GammaAdj['day']-int(CtyDayZero)+1

# CountyEffdistRecALL=CountyEffdistRecALL.merge(GammaAdj[['dayadj','CumGamma']],left_on='Days',right_on='dayadj',how='left')
GammaAdj=GammaAdj.fillna(0)
inpdata=TotRec[TotRec['DateStr']==datestr]
inpdata['x']=inpdata['ATD']
inpdata['x_low']=inpdata['ATD_low']
inpdata['x_high']=inpdata['ATD_high']
inpdata['ATD']=np.interp(inpdata['x'],GammaAdj['CumGamma'],GammaAdj['dayadj'])
inpdata['ATD_low']=np.interp(inpdata['x_low'],GammaAdj['CumGamma'],GammaAdj['dayadj'])
inpdata['ATD_high']=np.interp(inpdata['x_high'],GammaAdj['CumGamma'],GammaAdj['dayadj'])

inpdata=inpdata.merge(CtyLoc,left_on='FIPS',right_on='FIPS',how='left')
inpdata['text']=inpdata['Admin2']+','+inpdata['Province_State']+','+inpdata['FIPS'].astype(str)#+','+inpdata['Days'].astype(str)
inpdata['FIPS_zfill']=inpdata['FIPS'].astype(str).str.zfill(5)
from urllib.request import urlopen
import json
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)

    
import plotly.express as px

fig = px.choropleth(inpdata, geojson=counties, locations='FIPS_zfill', color='ATD',
                           color_continuous_scale='GnBu',#"Viridis",
                           range_color=(0, 15),
                           #scope="usa",
                           labels={'ATD':'Arrival Time Delay'}
                          )    
    

fig.update_layout(
        #title = 'Arrival time delay for every counties on '+datestr+' if national emergency declares seven days earlier',
        paper_bgcolor = 'rgba(0,0,0,0)',
        plot_bgcolor = 'rgba(0,0,0,0)',
        margin=dict(l=0, r=0, t=0, b=0),
        width=1300,height=700,
        geo_scope='usa',
        coloraxis_showscale=False
    )
#fig.update_traces(showscale=False)
fig.write_image('/Figure/FinalResult/ATD_Map/ATDMap_Adv_Fill407.pdf')
fig.write_image('/Figure/FinalResult/ATD_Map/ATDMap_Adv_Fill407.png')



In [ ]:
datestr='04-01'
Gamma=pd.read_csv('/Gamma_Dataset.csv',index_col=[0])
Gamma
GammaAdj=Gamma.copy()
# Cty day zero counts from 1/22
GammaAdj['CumGamma']=GammaAdj[GammaAdj['day']>=int(CtyDayZero)+1]['Gamma'].cumsum()

GammaAdj['dayadj']=GammaAdj['day']-int(CtyDayZero)+1

# CountyEffdistRecALL=CountyEffdistRecALL.merge(GammaAdj[['dayadj','CumGamma']],left_on='Days',right_on='dayadj',how='left')
GammaAdj=GammaAdj.fillna(0)
inpdata=TotRecDelay[TotRecDelay['DateStr']==datestr]
inpdata['x']=inpdata['ATD']
inpdata['x_low']=inpdata['ATD_low']
inpdata['x_high']=inpdata['ATD_high']
inpdata['ATD']=np.interp(inpdata['x'],GammaAdj['CumGamma'],GammaAdj['dayadj'])
inpdata['ATD_low']=np.interp(inpdata['x_low'],GammaAdj['CumGamma'],GammaAdj['dayadj'])
inpdata['ATD_high']=np.interp(inpdata['x_high'],GammaAdj['CumGamma'],GammaAdj['dayadj'])

inpdata=inpdata.merge(CtyLoc,left_on='FIPS',right_on='FIPS',how='left')
inpdata['text']=inpdata['Admin2']+','+inpdata['Province_State']+','+inpdata['FIPS'].astype(str)#+','+inpdata['Days'].astype(str)
inpdata['FIPS_zfill']=inpdata['FIPS'].astype(str).str.zfill(5)
from urllib.request import urlopen
import json
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)

    
import plotly.express as px

fig = px.choropleth(inpdata, geojson=counties, locations='FIPS_zfill', color='ATD',
                           color_continuous_scale='GnBu',#"Viridis",
                           range_color=(0, 15),
                           #scope="usa",
                           labels={'ATD':'Arrival Time Delay'}
                          )    
    

fig.update_layout(
        #title = 'Arrival time delay for every counties on '+datestr+' if national emergency declares seven days earlier',
        paper_bgcolor = 'rgba(0,0,0,0)',
        plot_bgcolor = 'rgba(0,0,0,0)',
        margin=dict(l=0, r=0, t=0, b=0),
        width=1300,height=700,
        geo_scope='usa',
        coloraxis_showscale=False
    )
#fig.update_traces(showscale=False)
fig.write_image('/Figure/FinalResult/ATD_Map/ATDMap_Delay_Fill407.pdf')
fig.write_image('/Figure/FinalResult/ATD_Map/ATDMap_Delay_Fill407.png')



In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl

fig,ax = plt.subplots(1,figsize=(9,0.5),dpi=100)
fig.subplots_adjust(bottom=0.5)

cmap = mpl.cm.GnBu
norm = mpl.colors.Normalize(vmin=0, vmax=15)

cb=fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),
             cax=ax, orientation='horizontal')
#fig.tight_layout()  # otherwise the right y-label is slightly clipped
cb.set_label(label='Arrival Time Delay', size=12, weight='bold')
cb.ax.tick_params(labelsize=12)
plt.savefig('/Figure/FinalResult/ATD_Map/ColorBar.pdf', format='pdf',bbox_inches='tight')
